<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2022-01-01" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Space X  Falcon 9 First Stage Landing Prediction**


## Hands on Lab: Complete the Machine Learning Prediction lab


Estimated time needed: **60** minutes


Space X advertises Falcon 9 rocket launches on its website with a cost of 62 million dollars; other providers cost upward of 165 million dollars each, much of the savings is because Space X can reuse the first stage. Therefore if we can determine if the first stage will land, we can determine the cost of a launch. This information can be used if an alternate company wants to bid against space X for a rocket launch.   In this lab, you will create a machine learning pipeline  to predict if the first stage will land given the data from the preceding labs.


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/landing_1.gif)


Several examples of an unsuccessful landing are shown here:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/crash.gif)


Most unsuccessful landings are planed. Space X; performs a controlled landing in the oceans.


## Objectives


Perform exploratory  Data Analysis and determine Training Labels

*   create a column for the class
*   Standardize the data
*   Split into training data and test data

\-Find best Hyperparameter for SVM, Classification Trees and Logistic Regression

*   Find the method that performs best using test data


## Import Libraries and Define Auxiliary Functions


In [ ]:
import piplite
await piplite.install(['numpy'])
await piplite.install(['pandas'])
await piplite.install(['seaborn'])

We will import the following libraries for the lab


In [ ]:
# Pandas is a software library written for the Python programming language for data manipulation and analysis.
import pandas as pd
# NumPy is a library for the Python programming language, adding support for large, multi-dimensional arrays and matrices, along with a large collection of high-level mathematical functions to operate on these arrays
import numpy as np
# Matplotlib is a plotting library for python and pyplot gives us a MatLab like plotting framework. We will use this in our plotter function to plot data.
import matplotlib.pyplot as plt
#Seaborn is a Python data visualization library based on matplotlib. It provides a high-level interface for drawing attractive and informative statistical graphics
import seaborn as sns
# Preprocessing allows us to standarsize our data
from sklearn import preprocessing
# Allows us to split our data into training and testing data
from sklearn.model_selection import train_test_split
# Allows us to test parameters of classification algorithms and find the best one
from sklearn.model_selection import GridSearchCV
# Logistic Regression classification algorithm
from sklearn.linear_model import LogisticRegression
# Support Vector Machine classification algorithm
from sklearn.svm import SVC
# Decision Tree classification algorithm
from sklearn.tree import DecisionTreeClassifier
# K Nearest Neighbors classification algorithm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report,  recall_score, accuracy_score, confusion_matrix, ConfusionMatrixDisplay

This function is to plot the confusion matrix.


In [ ]:
def plot_confusion_matrix(y,y_predict):
    "this function plots the confusion matrix"
    from sklearn.metrics import confusion_matrix

    cm = confusion_matrix(y, y_predict)
    ax= plt.subplot()
    sns.heatmap(cm, annot=True, ax = ax); #annot=True to annotate cells
    ax.set_xlabel('Predicted labels')
    ax.set_ylabel('True labels')
    ax.set_title('Confusion Matrix'); 
    ax.xaxis.set_ticklabels(['did not land', 'land']); ax.yaxis.set_ticklabels(['did not land', 'landed']) 
    plt.show() 

## Load the dataframe


Load the data


In [ ]:
from js import fetch
import io

URL1 = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/dataset_part_2.csv"
resp1 = await fetch(URL1)
text1 = io.BytesIO((await resp1.arrayBuffer()).to_py())
data = pd.read_csv(text1)

In [ ]:
data.head()

In [ ]:
data.shape

In [ ]:
data['LaunchSite'].unique()

In [ ]:
URL2 = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/dataset_part_3.csv'
resp2 = await fetch(URL2)
text2 = io.BytesIO((await resp2.arrayBuffer()).to_py())
X = pd.read_csv(text2)

In [ ]:
X.head(100)

## TASK  1


Create a NumPy array from the column <code>Class</code> in <code>data</code>, by applying the method <code>to_numpy()</code>  then
assign it  to the variable <code>Y</code>,make sure the output is a  Pandas series (only one bracket df\['name of  column']).


In [ ]:
Y = data.Class.to_numpy()  # or .values
Y

## TASK  2


Standardize the data in <code>X</code> then reassign it to the variable  <code>X</code> using the transform provided below.


In [ ]:
X.describe()

In [ ]:
# students get this 
transform = preprocessing.StandardScaler()
scaled_X = transform.fit_transform(X)
scaled_X

We split the data into training and testing data using the  function  <code>train_test_split</code>.   The training data is divided into validation data, a second set used for training  data; then the models are trained and hyperparameters are selected using the function <code>GridSearchCV</code>.


In [ ]:
pd.DataFrame(scaled_X).describe()

## TASK  3


Use the function train_test_split to split the data X and Y into training and test data. Set the parameter test_size to  0.2 and random_state to 2. The training data and test data should be assigned to the following labels.


<code>X_train, X_test, Y_train, Y_test</code>


In [ ]:
X_train, X_test, Y_train, Y_test  = train_test_split(scaled_X, Y, test_size=0.2, random_state = 2)

we can see we only have 18 test samples.


In [ ]:
Y_test.shape

## TASK  4


Create a logistic regression object  then create a  GridSearchCV object  <code>logreg_cv</code> with cv = 10.  Fit the object to find the best parameters from the dictionary <code>parameters</code>.


In [ ]:
parameters ={'C':[0.01,0.1,1],
             'penalty':['l2'],
             'solver':['lbfgs']}

In [ ]:
parameters ={"C":[0.01,0.1,1],'penalty':['l2'], 'solver':['lbfgs']}# l1 lasso l2 ridge
lr=LogisticRegression()
logreg_cv = GridSearchCV(estimator= lr, param_grid= parameters, cv=10, scoring='accuracy', verbose=0)
logreg_cv.fit(X_train, Y_train)

We output the <code>GridSearchCV</code> object for logistic regression. We display the best parameters using the data attribute <code>best_params\_</code> and the accuracy on the validation data using the data attribute <code>best_score\_</code>.


In [ ]:
print("tuned hpyerparameters :(best parameters) ",logreg_cv.best_params_)
print("accuracy :",logreg_cv.best_score_)

## TASK  5


Calculate the accuracy on the test data using the method <code>score</code>:


In [ ]:
accuracy_lr = logreg_cv.score(X_test, Y_test)
print(f"Test accuracy: {accuracy_lr:.4f}")

In [ ]:
# recall_score_LR = recall_score(y_test,y_pred, average='weighted')
# accuracy_score_LR = accuracy_score(y_test,y_pred)

Lets look at the confusion matrix:


In [ ]:
yhat_lr =logreg_cv.predict(X_test)
conf_matrix_lr = confusion_matrix(Y_test,yhat_lr)
plot_confusion_matrix(Y_test,yhat_lr)

In [ ]:
print(classification_report(Y_test,yhat_lr))

Examining the confusion matrix, we see that logistic regression can distinguish between the different classes.  We see that the problem is false positives.

Overview:

True Postive - 12 (True label is landed, Predicted label is also landed)

False Postive - 3 (True label is not landed, Predicted label is landed)


## TASK  6


Create a support vector machine object then  create a  <code>GridSearchCV</code> object  <code>svm_cv</code> with cv = 10.  Fit the object to find the best parameters from the dictionary <code>parameters</code>.


In [ ]:
parameters = {'kernel':('linear', 'rbf','poly','rbf', 'sigmoid'),
              'C': np.logspace(-3, 3, 5),
              'gamma':np.logspace(-3, 3, 5)}
svm = SVC()

In [ ]:
svm_cv = GridSearchCV(estimator= svm, param_grid= parameters, cv=10, scoring='accuracy', verbose=0)
svm_cv.fit(X_train, Y_train)

In [ ]:
print("tuned hpyerparameters :(best parameters) ",svm_cv.best_params_)
print("accuracy :",svm_cv.best_score_)

## TASK  7


Calculate the accuracy on the test data using the method <code>score</code>:


In [ ]:
accuracy_svm = svm_cv.score(X_test, Y_test)
print(f"Test accuracy: {accuracy_svm:.4f}")

We can plot the confusion matrix


In [ ]:
yhat_svm =svm_cv.predict(X_test)
conf_matrix_svm = confusion_matrix(Y_test,yhat_svm)
plot_confusion_matrix(Y_test,yhat_svm)

In [ ]:
print(classification_report(Y_test,yhat_svm))

## TASK  8


Create a decision tree classifier object then  create a  <code>GridSearchCV</code> object  <code>tree_cv</code> with cv = 10.  Fit the object to find the best parameters from the dictionary <code>parameters</code>.


In [ ]:
parameters = {'criterion': ['gini', 'entropy'],
     'splitter': ['best', 'random'],
     'max_depth': [2*n for n in range(1,10)],
     'max_features': ['log2', 'sqrt'],
     'min_samples_leaf': [1, 2, 4],
     'min_samples_split': [2, 5, 10]}

tree = DecisionTreeClassifier()

In [ ]:
tree_cv = GridSearchCV(estimator= tree, param_grid= parameters, cv=10, scoring='accuracy', verbose=0)
tree_cv.fit(X_train, Y_train)

In [ ]:
print("tuned hpyerparameters :(best parameters) ",tree_cv.best_params_)
print("accuracy :",tree_cv.best_score_)

## TASK  9


Calculate the accuracy of tree_cv on the test data using the method <code>score</code>:


In [ ]:
accuracy_tree = tree_cv.score(X_test, Y_test)
print(f"Test accuracy: {accuracy_tree:.4f}")

We can plot the confusion matrix


In [ ]:
yhat_tree = tree_cv.predict(X_test)
conf_matrix_tree = confusion_matrix(Y_test,yhat_tree)
plot_confusion_matrix(Y_test,yhat_tree)

In [ ]:
print(classification_report(Y_test,yhat_tree))

## TASK  10


Create a k nearest neighbors object then  create a  <code>GridSearchCV</code> object  <code>knn_cv</code> with cv = 10.  Fit the object to find the best parameters from the dictionary <code>parameters</code>.


In [ ]:
parameters = {'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
              'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
              'p': [1,2]}

KNN = KNeighborsClassifier()

In [ ]:

knn_cv = GridSearchCV(estimator= KNN, param_grid= parameters, cv=10, scoring='accuracy', verbose=0)
knn_cv.fit(X_train, Y_train)

In [ ]:
print("tuned hpyerparameters :(best parameters) ",knn_cv.best_params_)
print("accuracy :",knn_cv.best_score_)

## TASK  11


Calculate the accuracy of knn_cv on the test data using the method <code>score</code>:


In [ ]:
accuracy_knn = knn_cv.score(X_test, Y_test)
print(f"Test accuracy: {accuracy_knn:.4f}")

We can plot the confusion matrix


In [ ]:
yhat_knn = knn_cv.predict(X_test)
conf_matrix_knn = confusion_matrix(Y_test,yhat_knn)
plot_confusion_matrix(Y_test,yhat_knn)

In [ ]:
print(classification_report(Y_test,yhat_knn))

## TASK  12


Find the method performs best:


In [ ]:

print( logreg_cv.best_score_)
print( svm_cv.best_score_)
print( tree_cv.best_score_)
print( knn_cv.best_score_)      # Cross-Calidation Accuracy Score

This gives the mean **Cross-Calidation Accuracy Score** from your GridSearchCV (or cross_val_score) across all training folds during tuning.
Interpretation:

“How well did the model perform during cross-validation on the training set?”

Used for model selection, especially during hyperparameter tuning.

Based on internal training folds, not the holdout test set.

In [ ]:
print(accuracy_lr)      
print(accuracy_svm)         # Accuracy
print(accuracy_tree)
print(accuracy_knn)                  # eg knn_cv.score(X_test, Y_test)

This evaluates the best model found by GridSearchCV on the unseen test data using .score(), which for classifiers = **accuracy by default.**

Interpretation:

“How well does the tuned model generalize to the test set?”

This is your true evaluation of performance on new data.
**More realistic measure of generalization than best_score_.**

In [ ]:
print (accuracy_score(Y_test, yhat_lr))
print (accuracy_score(Y_test, yhat_svm))
print (accuracy_score(Y_test, yhat_tree))
print (accuracy_score(Y_test, yhat_knn))

accuracy_score(y_test, y_pred),  which requires you to import "from sklearn.metrics import accuracy_score" (also classification_report, confusion_matrix, ConfusionMatrixDisplay, recall_score etc.) computes accuracy manually from predicted(**yhat**) and true labels(**Y_test**).

Interpretation:

“What proportion of predictions were correct?”

**Same as .score()** but gives you more control you can swap in other metrics.

You need to manually call .predict() beforehand. **eg yhat_knn = knn_cv.predict(X_test)**

In [ ]:
recall_lr = recall_score(Y_test,yhat_lr)   # True Positive
recall_svm = recall_score(Y_test,yhat_svm)
recall_tree = recall_score(Y_test,yhat_tree)
recall_knn = recall_score(Y_test,yhat_knn)
print(recall_lr,"\n", recall_svm,"\n", recall_tree,"\n", recall_knn)

In [ ]:
recall_FalseP_lr = recall_score(Y_test, yhat_lr, average = None)[0]   # False Positve
recall_FalseP_svm = recall_score(Y_test, yhat_svm, average = None)[0]
recall_FalseP_tree = recall_score(Y_test, yhat_tree, average = None)[0]
recall_FalseP_knn = recall_score(Y_test, yhat_knn, average = None)[0]


In [ ]:
recall_FalseP_knn

In [ ]:
conf_matrix_tree

In [ ]:
correctP_lr = conf_matrix_lr[0,0]+conf_matrix_lr[1,1]            #No of Correct Predictions	
correctP_svm = conf_matrix_svm[0,0]+conf_matrix_svm[1,1]
correctP_tree = conf_matrix_tree[0,0]+conf_matrix_tree[1,1]
correctP_knn = conf_matrix_knn[0,0]+conf_matrix_knn[1,1]

In [ ]:
best_model_df = pd.DataFrame(index = ['Accuracy %', 'Cross-Validation Accuracy Score', 'No of Correct Predictions' , 'TruePositive %', 'FalsePositive %'], columns = ['Logistic Regression', 'SVM', 'Decision Tress', 'KNN'])
best_model_df

In [ ]:
accuracy  = np.array([accuracy_lr, accuracy_svm,  accuracy_tree, accuracy_knn])
correctpredictions  = np.array([correctP_lr, correctP_svm, correctP_tree, correctP_knn])
Truepositve_recall =  np.array([recall_lr, recall_svm, recall_tree, recall_knn])
FalsePositive_recall1 = np.array([recall_FalseP_lr, recall_FalseP_svm, recall_FalseP_tree, recall_FalseP_knn])
CV_acc_Score = np.array([logreg_cv.best_score_, svm_cv.best_score_, tree_cv.best_score_, knn_cv.best_score_])


In [ ]:
best_model_df.loc['Accuracy %'] = np.round(accuracy*100, 2)
best_model_df.loc['Cross-Validation Accuracy Score'] = np.round(CV_acc_Score*100,2)
best_model_df.loc['No of Correct Predictions'] = correctpredictions
best_model_df.loc['TruePositive %'] = np.round(Truepositve_recall*100,2)
best_model_df.loc['FalsePositive %'] = np.round(FalsePositive_recall1*100,2)


In [ ]:
best_model_df

In [ ]:
acc_bestmodel = best_model_df.loc['Accuracy %']
acc_bestmodel = pd.DataFrame(acc_bestmodel).reset_index()
acc_bestmodel = acc_bestmodel.rename(columns={'index': 'Model'})

In [ ]:
acc_bestmodel

In [ ]:
ax = sns.barplot(acc_bestmodel, y= 'Model', x='Accuracy %' )

# Add labels on top of each bar
for i, value in enumerate(acc_bestmodel['Accuracy %']):
        ax.text(value + 3, i, f"{value:.0f} %", va='center', fontweight='bold', color='red')


plt.title("Space X Falcon 9 First Stage Landing Prediction \n \n Accuracy Rate of Pridictive Models with thier Tuned Hyperparameters\n")
plt.xlabel('Accracy rate')
plt.xlim(0,100)
plt.ylabel('')
plt.show()

## Authors


[Pratiksha Verma](https://www.linkedin.com/in/pratiksha-verma-6487561b1/)


<!--## Change Log--!>


<!--| Date (YYYY-MM-DD) | Version | Changed By      | Change Description      |
| ----------------- | ------- | -------------   | ----------------------- |
| 2022-11-09        | 1.0     | Pratiksha Verma | Converted initial version to Jupyterlite|--!>


### <h3 align="center"> IBM Corporation 2022. All rights reserved. <h3/>
